# 銷貨預測模型_MA3&迴歸
## 主要撰寫者：陳宇鑫
## 說明：
## 1. 如果不需要做數據預處理，只需先跑前兩個Kernal，之後跳到「存取數據預處理完的資料集」的部分讀取處理好的資料集後再往下繼續執行 (但是後來經過驗證後發現to_csv會使小數儲存的部分出現問題；無法完整保留特徵縮放後資料的所有小數位，會喪失最後幾位。這樣會導致之後線性迴歸模型和相關係數的結果有些微的落差，因此建議還是從頭開始跑該程式碼)
## 2. 標題標上**的Kernal代表在本次的專案中完全不會執行到該部分

## 匯入所需的函式庫並指定建模所使用的GPU

In [1]:
# 數據操作和計算相關的函式庫
import pandas as pd 
import numpy as np
import math

# 繪圖相關的函式庫
import matplotlib.pyplot as plt

# IPython顯示相關的函式庫
from IPython.display import display_html

# 機器學習相關的函式庫
from sklearn import preprocessing # sklearn數據預處理的函式庫

# 系統相關的函式庫
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 指定建模所使用的GPU

## 系統相關設定

In [2]:
# 設定dataframe在輸出時可以看到所有的欄位
pd.set_option('display.max_columns', None)

# 設定dataframe顯示到小數第四位
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 讓plot能夠顯示中文字體
plt.rcParams['font.sans-serif'] = ['simhei']

## 取得原始資料集

In [3]:
data = pd.read_csv("no_lag_model_data.csv", index_col = 0)
#data.head() # 大致看一下讀進來的原始資料集

## 從原始資料集中 篩選出之後會用到 / 去掉之後不會用到 的特定欄位

In [4]:
data = data.drop(['生管課'], axis = 1) # 拿掉「生管課」的欄位，因為該欄位全部的資料為0，對預測毫無幫助

# 數據預處理

## 數據預處理 - 前置作業

### 將月份的獨熱編碼重新命名 (原本只用01、02...這種數字表示會讓人不知道這是月份的獨熱編碼)

In [5]:
data = data.rename(columns = {"01": "1月", "02": "2月", "03": "3月", "04": "4月", "05": "5月", "06": "6月", 
                              "07": "7月", "08": "8月", "09": "9月", "10": "10月", "11": "11月", "12": "12月"})

## 數據預處理 - 對個別產品的特徵進行MinMax特徵縮放 

### 先額外生成一個「實銷貨量_原始值」的欄位，以保留實銷貨量在進行特徵縮放之前的值

In [6]:
data.insert(2, '實銷貨量_原始值', data['實銷貨量']) # 在data中index = 2的欄位插入自「實銷貨量」複製的欄位「實銷貨量_原始值」
#data.head() # 查看處理後的資料集是否有異常

### 以下兩個Kernal選擇其中一個執行

### 選項(1)：對「個別產品」進行MinMax特徵縮放

In [7]:
### 對個別產品作minmax ###
startNormColumnIndex = data.columns.get_loc('實銷貨量') # 從哪一個欄位開始進行特徵縮放的索引
trainingPeriod = 47 # 訓練集的期間數

normX_List = [None] * len(data['產品名稱'].unique()) # 用來儲存各個產品特徵縮放後的資料

# 開始進行個別產品特徵資料的縮放
for idx, p_name in enumerate(list(data['產品名稱'].unique())):

    X = data[data["產品名稱"] == p_name][data.columns[startNormColumnIndex:]] # 取出個別產品的資料
    X_trainingPeriod = X.iloc[:trainingPeriod] # 取出個別產品訓練期間的資料

    scaler = preprocessing.MinMaxScaler().fit(X_trainingPeriod) # 用個別產品訓練期間的資料去配適縮放器
    
    normX = pd.DataFrame(scaler.transform(X), columns = X.columns) # 用配適好的縮放器對個別產品整體的資料進行特徵縮放
    
    normX_List[idx] = normX # 將縮放後的結果儲存至清單中
    
normX_allProduct = pd.concat(normX_List, ignore_index = True) # 合併所有產品特徵縮放後的資料

data = pd.concat([data[data.columns[:startNormColumnIndex]], normX_allProduct], axis = 1) # 合併沒有和有特徵縮放的變數

#data # 查看縮放後的結果有無異常

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

...

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/isvms/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with i

### 選項(2)：對「整體產品」進行MinMax特徵縮放

In [ ]:
# 用訓練集期間的資料去配適縮放器
scaler = preprocessing.MinMaxScaler().fit(
    data[~data['年/月'].isin(['2017/12'])].iloc[:, data.columns.get_loc('實銷貨量'):]
)

# 用配適好的縮放器對整體的資料進行特徵縮放
data.iloc[:, data.columns.get_loc('實銷貨量'):] = scaler.transform(data.iloc[:, data.columns.get_loc('實銷貨量'):])

data # 查看縮放後的結果有無異常

## 數據預處理 - 新增延遲欄位
### 說明：目前是用前三期的資料去預測下一期的資料，例如如果現在是7月，要預測下一個月(8月)的資料，就要拿4~6月的資料去預測，因此Y(8月)要對到的X是從Y的前「兩」期開始(6月)，往後延遲三期的資料(6、5、4月的資料)

In [8]:
lagFromWhichLastPeriod = 2 # 要從前幾期的資料開始延期
maxLagPeriod_num = 3 # 最大延遲的期數

firstVarNeededtoLag_name = '實銷貨量' # 要開始進行延遲的變數名稱

firstVarNeededtoLag_index = data.columns.get_loc(firstVarNeededtoLag_name) # 取得要開始進行延遲的變數的欄位索引

# 按照之前設定的參數生成各個變數的延遲欄位
for idx, col_name in enumerate(data.columns):
    if (idx >= firstVarNeededtoLag_index):  # 判斷這個變數是否為要生成延遲欄位的變數      
        # 開始進行生成該變數延遲欄位的迭代(如果要延遲三期就要跑三次)
        for i in range(lagFromWhichLastPeriod, maxLagPeriod_num + lagFromWhichLastPeriod):
            new_col_name = col_name + '_延遲' + str(i) + '期' # 生成該變數延遲欄位的名稱
            data[new_col_name] = data[col_name].shift(i) # 生成該變數的延遲欄位

#data # 查看處理後的資料集是否有異常

## **數據預處理 - 新增平方、交互作用項 
### 說明：
### 1. 目前最高次數為2(degree = 2)，即包含特徵的平方項和兩兩特徵相乘的交互作用項
### 2. 先不在這裡加上截距項(include_bias = False)；之後建模時會自動加上

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 建立多項式生成器
poly = PolynomialFeatures(
    degree = 2, 
    interaction_only = False, # 是否只要有交互作用項(設成False會另外生成特徵的平方項)
    include_bias = False # 是否要新增一個全為1的欄位
)

# 生成特徵的多項式資料
featuresAndTheirPolyFeatures = poly.fit_transform(data.iloc[:,data.columns.get_loc('實銷貨量_延遲2期'):])

# 將結果建立成一個dataframe
data_featuresAndTheirPolyFeaturesOnly = pd.DataFrame(
    data = featuresAndTheirPolyFeatures,
    columns = poly.get_feature_names(data.iloc[:,data.columns.get_loc('實銷貨量_延遲2期'):].columns)
)

data = data.reset_index(drop = True) # 重設data的index(從0開始往下排)

# 將特徵和其交互作用項和原本的資料合併
data = pd.concat(
    [data.iloc[:, :data.columns.get_loc('12月')], data_featuresAndTheirPolyFeaturesOnly], 
    axis = 1
)

#data.head() # 查看處理後的資料集是否有異常

## 存取數據預處理完的資料集

### 儲存

In [ ]:
data.to_csv('data_for_reg_0827_allProduct_afterMinMaxByEachProduct_lag2To4Period.csv', index = False)

### 讀取

In [ ]:
data = pd.read_csv('data_for_reg_0827_allProduct_afterMinMaxByEachProduct_lag2To4Period.csv')
print(data.shape) # 查看讀進來的資料維度

## (optional) 去除特定的變數

In [ ]:
# # 內部變數
# data = data[data.columns.drop(list(data.filter(regex='月_')))]

# 外部變數
data = data[data.columns.drop(list(data.filter(regex='物價指數')))]
data = data[data.columns.drop(list(data.filter(regex='食品、飲料及菸草製品零售業之營業額')))]
data = data[data.columns.drop(list(data.filter(regex='這個月或下個月是否是')))]

#data.head() # 查看處理完的資料

## 如果要用MinMax縮放後實銷貨量作為Y_train，必須要先運行這一個部分儲存各個產品Y_trian中的最大和最小值，之後才能將預測值還原

In [9]:
# 建立用來儲存各個產品Y_trian中的最大和最小值的清單
Y_train_max_by_product_list = [] 
Y_train_min_by_product_list = []

oneProductPeriodNum_train = 47 # 訓練集中個別產品共有多少期數的資料
oneProductPeriodNum = 48 # 個別產品共有多少期數的資料

norm_Y_train_List = [None] * len(data['產品名稱'].unique()) # 用來儲存各個產品特徵縮放後的資料

for i in range(len(data['產品名稱'].unique())):
    
    # 取出各個產品的Y_train
    Y_train_oneProduct = (
        data['實銷貨量_原始值'][oneProductPeriodNum * i : oneProductPeriodNum * i + oneProductPeriodNum_train]
    )
    
    Y_train_max_by_product_list.append(np.max(Y_train_oneProduct)) # 儲存該產品Y_train的最大值
    
    Y_train_min_by_product_list.append(np.min(Y_train_oneProduct)) # 儲存該產品Y_train的最小值

## 拿掉之前因為生成延遲變數導致出現遺漏值的資料 
### 說明：目前是用各產品前三期的資料去預測下一期實銷貨量，以該資料為例，即用各產品2014年1~3月的資料預測5月的實銷貨量，2~4月的資料預測6月的實銷貨量，以此類推，因此在生成延遲變數後，前四期的資料，即各產品2014/01~2014/04的資料，會出現遺漏值(例如2014/04的實銷貨量要對到2013/12~2014/02的特徵，但資料從2014/01開始，不會有2013/12的資料)，需要拿掉

In [10]:
lagFromWhichLastPeriod = 2 # 要從前幾期的資料開始延期
maxLagPeriod_num = 3 # 最大延遲的期數

for i in range(1, maxLagPeriod_num + 2):
    if i < 10:
        data = data.drop(data[(data['年/月'] == ('2014/0' + str(i)))].index)
    else:
        data = data.drop(data[(data['年/月'] == ('2014/' + str(i)))].index)
        
data = data.reset_index(drop = True) # 重設dataframe的索引(從0開始往下排)

#data # 查看處理完的資料

## 建立訓練集(測試集以外期間的資料)和測試集(2017年12月的資料)

In [11]:
# 建立訓練集
X_train = data[~data['年/月'].isin(['2017/12'])].reset_index(drop = True).iloc[:,data.columns.get_loc('實銷貨量_延遲2期'):]
Y_train = data[~data['年/月'].isin(['2017/12'])].reset_index(drop = True)['實銷貨量']

# 建立測試集
X_test = data[data['年/月'].isin(['2017/12'])].reset_index(drop = True).iloc[:,data.columns.get_loc('實銷貨量_延遲2期'):]
Y_test = data[data['年/月'].isin(['2017/12'])].reset_index(drop = True)['實銷貨量_原始值']

# 檢查訓練集和測試集的維度是否有異常
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(312051, 180)
(312051,)
(7257, 180)
(7257,)


## 探索性分析：列出之後要放入模型中的解釋變數，並查看這些變數和目標變數(實銷貨量)之間的相關係數

### 相關函式

In [16]:
"""
### display_side_by_side ###
函式用途：可以並排顯示dataframe的函式
參考來源：https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

Parameters:
1. *args
    可以是好幾個不同的dataframe
2. showIndex: boolean
    是否要在dataframe中顯示索引
"""
from IPython.display import display_html
def display_side_by_side(*args, showIndex = True):
    html_str=''
    for df in args:
        html_str+=df.to_html(index = showIndex)
    display_html(html_str.replace('table','table style="display:inline"'), raw = True)

    
"""
### displayFeature ###
函式用途：列出放入模型中的所有解釋變數(特徵)，並將結果平攤展示(避免變數過多導致最後的dataframe太長)

Parameters:
1. featureName_list: list
    存有特徵名稱的清單
2. colNum = 3: int
    要把解釋變數平攤成幾欄位顯示
"""    
def displayFeature(featureName_list, colNum = 3):
    
    df_featureName_list = [] # 用來裝解釋變數分段結果的清單
    
    rowLen = math.ceil(len(featureName_list) // colNum) # 算出平攤後的欄位長度
    
    # 開始對解釋變數進行分段，並將分段結果儲存在清單中
    for i in range(colNum):
        if i < (colNum - 1):
            df_featureName_list.append(
                pd.DataFrame(featureName_list[i * rowLen: (i + 1) * rowLen], columns = ['解釋變數_第' + str(i + 1) +'部分'])
            )
        # 最後一欄的最後幾個row可能會是沒有資料的，因此在設定取資料的索引時不能設end的值，要獨立出來處理
        # 例如原來有40個變數，平分成3欄後每欄的長度為math.ceil(40 / 3) = 14，然而這樣最後一欄只會放12而不是14個變數，會空出2格
        else: 
            df_featureName_list.append(
                pd.DataFrame(featureName_list[i * rowLen:], columns = ['解釋變數_第' + str(i + 1) +'部分'])
            )
            
    # 將裝有分段結果的清單丟到可以並排顯示dataframe的函式處理，並設定不讓dataframe顯示索引(這裡的索引只是沒有實質意義的數字)
    display_side_by_side(*df_featureName_list, showIndex = False) 
    
    
"""
### displayTargetVersusOtherVarCorr ###
函式用途：計算目標變數和各個解釋變數的相關係數，並將結果平攤展示(避免變數過多導致最後的dataframe太長)

Parameters:
1. X: array
    特徵資料
2. Y: array
    目標變數資料
3. targetName: str
    目標變數的名稱
4. method: {'pearson', 'kendall', 'spearman'}
    算相關係數的方法，一共有三種，預設是'pearson'
5. colNum: int
    要將結果平攤成幾欄位顯示
"""     
def displayTargetVersusOtherVarCorr(X, Y, targetName = '目標變數', method = 'pearson', colNum = 3):
    
    # 把X和Y合併，以方便用pandas的corr計算相關係數矩陣
    XandY = X.copy()
    XandY[targetName] = Y
    
    featureVersusTargetCorr = pd.DataFrame(XandY.corr(method = method)[targetName]) # 取得目標變數和各個解釋變數的相關係數

    df_featureVersusTargetCorr_list = [] # 用來裝目標變數和各個解釋變數的相關係數分段結果的清單
    
    rowLen = math.ceil(len(featureVersusTargetCorr)) // 3 # 算出平攤後的欄位長度
    
    # 開始對目標變數和各個解釋變數的相關係數進行分段，並將分段結果儲存在清單中
    for i in range(colNum): # 要將結果平攤成多少欄位顯示就要跑多少次的迭代
        if i < (colNum - 1):
            df_featureVersusTargetCorr_list.append(
                pd.DataFrame(featureVersusTargetCorr[i * rowLen: (i + 1) * rowLen])
            )
        else: # 最後一欄的最後幾個row可能會是沒有資料的，因此在設定取資料的索引時不能設end的值，要獨立出來處理
            df_featureVersusTargetCorr_list.append(
                pd.DataFrame(featureVersusTargetCorr[i * rowLen:])
            )
    
    # 將裝有分段結果的清單丟給可以並排顯示dataframe的函式處理 
    display_side_by_side(*df_featureVersusTargetCorr_list) 

### 列出之後要放入模型中的解釋變數(特徵)

In [19]:
displayFeature(list(X_train), colNum = 10)

解釋變數_第1部分
實銷貨量_延遲2期
實銷貨量_延遲3期
實銷貨量_延遲4期
含稅總成本_延遲2期
含稅總成本_延遲3期
含稅總成本_延遲4期
成本合計_延遲2期
成本合計_延遲3期
成本合計_延遲4期
銷貨淨額_延遲2期


### Pearson 相關係數

In [ ]:
displayTargetVersusOtherVarCorr(X_train, Y_train, targetName = '實銷貨量')

### Kendall 相關係數

In [ ]:
displayTargetVersusOtherVarCorr(X_train, Y_train, targetName = '實銷貨量', method = 'kendall')

### Spearman 相關係數

In [ ]:
displayTargetVersusOtherVarCorr(X_train, Y_train, targetName = '實銷貨量', method = 'spearman')

# 模型建置

## 建置模型相關函式

In [12]:
"""
### displayModelCoef ###
函式用途：展示模型中各個解釋變數的迴歸係數(包含截距項)

Parameters:
1. modelName: str 
    預測模型的名稱
2. varNameList: list
    該模型放入的變數名稱清單
3. model_intercept: array
    該模型的截距項
4. model_coef: array
    該模型中各個解釋變數的迴歸係數
5. sortCoef: boolean
    是否要將dataframe中的變數按照迴歸係數的大小降冪排列
"""
def displayModelCoef(modelName, varNameList, model_intercept, model_coef, sortCoef = False):
    
    # 將截距項和各個解釋變數的迴歸係數合併，並建立成一個dataframe
    featureName = np.append('截距項', varNameList)
    coef = np.append(model_intercept, model_coef)
    df_modelCoef = pd.DataFrame(data = {'變數名稱': featureName, (modelName + '_迴歸係數'): coef})
    
    # 如果sortCoef設定為True則將dataframe中的解釋變數按照迴歸係數的大小降冪排列
    if sortCoef == True:
        df_modelCoef = df_modelCoef.sort_values(by = (modelName + '_迴歸係數'), ascending = False) 
    
    # 讓系統能在這一個部分顯示dataframe完整的row
    with pd.option_context("display.max_rows", df_modelCoef.shape[0]): 
        display(df_modelCoef)
        
        
"""
### inverseMinMaxByProduct ###
函式用途：如果之前對某個欄位by產品進行縮放，並有同時儲存該欄位by產品縮放的最大和最小值清單，這個函式能將該欄位還原成縮放前的尺度

Parameters:
1. data: 1d-array 
    一維的資料(向量)，例如df中的一個欄位
2. data_max_by_product_list: list
    該資料by產品的最大值清單
3. data_min_by_product_list: list
    該資料by產品的最小值清單
"""
def inverseMinMaxByProduct(data, data_max_by_product_list, data_min_by_product_list):

    data_inverseMinMax = [] # 儲存還原後的資料
    oneProductPeriod = data.shape[0] // len(data_max_by_product_list) # 計算一個產品會有幾期的資料
    product_index = -1 # 產品的索引

    for idx, val in enumerate(data):
        
        # 如果該判斷式成立，代表要切換產品做資料還原了，因此將產品的索引增加1，即下一個產品的索引
        if idx % oneProductPeriod == 0:
            product_index += 1
        
        # 將縮放後的資料還原，並存入清單中 (MinMax還原的方法：val * (Min - Max) + Min)
        data_inverseMinMax.append( 
            val * (
                data_max_by_product_list[product_index] - data_min_by_product_list[product_index]
            ) + data_min_by_product_list[product_index]
        )
        
    return data_inverseMinMax # 回傳還原後的資料


"""
### displayForecastAccuracy ###
函式用途：展示模型預測測試集成效的各項指標(MSE、調整後r平方、MAPE)

Parameters:
1. modelName: str 
    預測模型的名稱
2. Y_real: array
    Y的實際值
3. Y_pred: array
    Y的預測值
4. X_test: array
    測試集的特徵資料
"""
def displayForecastAccuracy(modelName, Y_real, Y_pred, X_test):

    # 根據Y的實際值和預測值計算MSE和調整後r平方
    mse = np.mean((Y_real - Y_pred) ** 2)
    r_squared = 1 - sum((Y_real - Y_pred) ** 2) / sum((Y_real - np.mean(Y_real)) ** 2) 
    #adj_r_squared = r_squared - (1 - r_squared) * (X_test.shape[1] / (X_test.shape[0] - X_test.shape[1] - 1))

    adj_r_squared = 1 - (X_test.shape[0] - 1) * (1 - r_squared) / (X_test.shape[0] - X_test.shape[1])
    # adj_r_squared = 1 - (n - 1) * (1 - r_squared) / (n - p)

    # 將Y的實際值和預測值建立成名稱為stat的dataframe
    stat = pd.DataFrame()
    stat['real'] = Y_real
    stat['pred'] = Y_pred

    # 由於測試集中的實銷貨量會有0和接近0的值，這樣會使MAPE算不出來或是暴增，因此我們在算MAPE時會對stat做以下的調整：
    # 1. 測試集中的實銷貨量(Y的實際值；stat['real'])範圍界在0到1之間設成1；-1到0之間的設成-1。
    # 2. 實銷貨量預測值(Y的預測值；stat['pred'])範圍界在0到1之間設成1；-1到0之間的設成-1。
    stat_adj = stat.copy()
    stat_adj.loc[(stat_adj['real'] > -1) & (stat_adj['real'] < 0), 'real'] = -1
    stat_adj.loc[(stat_adj['real'] <  1) & (stat_adj['real'] >=0), 'real'] = 1
    stat_adj.loc[(stat_adj['pred'] > -1) & (stat_adj['pred'] < 0), 'pred'] = -1
    stat_adj.loc[(stat_adj['pred'] <  1) & (stat_adj['pred'] >=0), 'pred'] = 1
    stat_adj['mape'] = np.abs((stat_adj['real'] - stat_adj['pred']) / stat_adj['real']) * 100

    # 根據MAPE對stat做升冪排序，並將索引重新排序
    stat_adj_sort = stat_adj.copy()
    stat_adj_sort = stat_adj_sort.sort_values(by = ['mape'], ascending = True)
    stat_adj_sort = stat_adj_sort.reset_index(drop = True) # 把索引重新排序

    # 計算測試集整體資料的MAPE，以及誤差排序後最低的75%、50%、20%、10%資料的MAPE
    mape_all = stat_adj_sort['mape'].mean()
    mape_75 = stat_adj_sort['mape'].iloc[0:(round(len(stat_adj_sort)*0.75))].mean()
    mape_50 = stat_adj_sort['mape'].iloc[0:(round(len(stat_adj_sort)*0.50))].mean()
    mape_20 = stat_adj_sort['mape'].iloc[0:(round(len(stat_adj_sort)*0.20))].mean()
    mape_10 = stat_adj_sort['mape'].iloc[0:(round(len(stat_adj_sort)*0.10))].mean()

    # 統整各項預測成效指標至dataframe當中
    df_performanceIndicatorsOfPrediction = pd.DataFrame(
        data = [mse, adj_r_squared, mape_all, mape_75, mape_50, mape_20, mape_10],
        index = ['MSE', 'Adj R-Squared', '整體資料的MAPE', '誤差排序後最低的75%資料的MAPE', '誤差排序後最低的50%資料的MAPE', 
                 '誤差排序後最低的20%資料的MAPE', '誤差排序後最低的10%資料的MAPE'],
        columns = [modelName]
    )
    
    display(df_performanceIndicatorsOfPrediction) # 展示處理完的結果
    
    
"""
### draw_forecast_MAPE_distribution ###
函式用途：畫出模型預測測試集的MAPE分布

Parameters:
1. modelName: str 
    預測模型的名稱
2. Y_real: array
    Y的實際值
3. Y_pred: array
    Y的預測值
4. classCap: int
    分組的上限值，之後大於該值的部分會以"classCap+"來表示
5. classInterval: int
    每一組的間隔大小
備註：classCap最好要能整除classInterval，例如(25000, 5000)，而且相除的商數推薦在10以下，避免因組別太多而降低圖片的呈現品質

## 範例 ##
如果將classCap設成25000；classInterval設成5000，將會有七個組別：
            MAPE範圍
    第一組  0 (等於0)
    第二組  0~5000 (大於0且小於等於5000)
    第三組  5000~10000
    ...
    第六組  20000~25000
    最七組  25000+
"""
def draw_forecast_MAPE_distribution(modelName, Y_real, Y_pred, classCap, classInterval):
    
    # 將Y的實際值和預測值建立成名稱為stat的dataframe
    stat = pd.DataFrame()
    stat['real'] = Y_real
    stat['pred'] = Y_pred

    # 由於測試集中的實銷貨量會有0和接近0的值，這樣會使MAPE算不出來或是暴增，因此我們在算MAPE時會對stat做以下的調整：
    # 1. 測試集中的實銷貨量(Y的實際值；stat['real'])範圍界在0到1之間設成1；-1到0之間的設成-1。
    # 2. 實銷貨量預測值(Y的預測值；stat['pred'])範圍界在0到1之間設成1；-1到0之間的設成-1。
    stat_adj = stat.copy()
    stat_adj.loc[(stat_adj['real'] > -1) & (stat_adj['real'] < 0), 'real'] = -1
    stat_adj.loc[(stat_adj['real'] <  1) & (stat_adj['real'] >=0), 'real'] = 1
    stat_adj.loc[(stat_adj['pred'] > -1) & (stat_adj['pred'] < 0), 'pred'] = -1
    stat_adj.loc[(stat_adj['pred'] <  1) & (stat_adj['pred'] >=0), 'pred'] = 1
    stat_adj['mape'] = np.abs((stat_adj['real'] - stat_adj['pred']) / stat_adj['real']) * 100
    
    # 建立MAPE分組資料和其標籤名稱的清單
    relFreq_MAPE_list = []
    labelName_list = []
    
    # 取得第一組「MAPE等於0」的相對頻率和其標籤名稱，並儲存在清單中
    relFreq_MAPE_list.append(stat_adj[stat_adj['mape'] == 0]['mape'].count() / len(Y_real))
    labelName_list.append('0')
    
    # 取得各組「MAPE介於某個範圍」的相對頻率和其標籤名稱，並儲存在清單中
    for i in range(0, classCap, classInterval):
        relFreq_MAPE_list.append(
            stat_adj[(stat_adj['mape'] > i) & (stat_adj['mape'] <= i + classInterval)]['mape'].count() / len(Y_real)
        )
        labelName_list.append(str(i) + '~' + str(i + classInterval))
    
    # 取得最後一組「MAPE大於分組上限值」的相對頻率和其標籤名稱，並儲存在清單中
    relFreq_MAPE_list.append(stat_adj[stat_adj['mape'] > classCap]['mape'].count() / len(Y_test))
    labelName_list.append(str(classCap) + '+')

    # 把MAPE各組的相對頻率和其標籤名稱建立成一個dataframe
    df_relFreq_MAPE = pd.DataFrame(
        {'lab': labelName_list, 
         'val': relFreq_MAPE_list}
    )

    # 根據MAPE的分組結果畫MAPE分布的長條圖
    ax = df_relFreq_MAPE.plot.bar(
        figsize = (15, 7),
        fontsize = 16,
        x = 'lab', 
        y = 'val', 
        rot = 0,
        legend = False
    )
    
    # 其他和圖片相關的設定
    ax.set_title(('「' + modelName + '」預測測試集的MAPE分布'), fontsize = 20)
    ax.set_xlabel('MAPE範圍', fontsize=20)
    ax.set_ylabel('相對頻率', fontsize=20)
    
    # 在長條圖中的各個長條上顯示該長條對應的Y值
    for i, v in enumerate(df_relFreq_MAPE['val'].values):
        ax.text(i, v, round(v, 3), horizontalalignment='center', fontsize = 20)

## 模型0 - MA3 (用前三期的資料去預測下一期的資料)

### 生成一個dataframe包含測試集中各產品的實銷貨量(原始值)和其MA3的預測值
### 說明：在本案例中MA3即是用各產品2017/8~10實銷貨量的平均去預測各產品在2017/12(測試集)的資料

In [ ]:
# 生成一個dataframe包含「年/月」、「實銷貨量_原始值」和「將「實銷貨量_原始值」的三期移動平均數延遲兩期(MA3預測值)」的欄位
data_salesVolumn = pd.DataFrame(
    {'年/月': data['年/月'].copy(),
     '實銷貨量_原始值': data['實銷貨量_原始值'].copy(),
     '實銷貨量_原始值_MA3_延遲2期': data["實銷貨量_原始值"].copy().rolling(window = 3).mean().shift(2)
    }
)

# 取出符合測試集期間的資料
data_salesVolumn = data_salesVolumn[data_salesVolumn['年/月'].isin(['2017/12'])].reset_index(drop = True)

#data_salesVolumn.head() # 查看建好的dataframe

### 展示模型預測測試集成效的各項指標

In [ ]:
modelName = 'MA3' # 模型名稱

## 取得Y的實際值和預測值 ##
Y_real = data_salesVolumn['實銷貨量_原始值'].values
Y_pred = data_salesVolumn['實銷貨量_原始值_MA3_延遲2期'].values

# X_test_MA3即MA3預測值，這裡改變MA3預測值的維度((7257, ) → (7257, 1))是為了在之後計算調整後的R-Squared
X_test_MA3 = np.asarray(data_salesVolumn['實銷貨量_原始值_MA3_延遲2期']).reshape(-1, 1) 

displayForecastAccuracy(modelName, Y_real, Y_pred, X_test_MA3) # 展示模型預測測試集成效的各項指標
draw_forecast_MAPE_distribution(modelName, Y_real, Y_pred, 25000, 5000) # 畫出模型預測測試集的MAPE分布

## 模型1 - Linear Regression

### 配適模型

In [13]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression() # 設定regressor
linreg.fit(X_train, Y_train) # 配適模型

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

### 用statsmodels.api的OLS建模可以看解釋變數個別t檢定的p值

In [14]:
import statsmodels.api as sm 
X_train2 = sm.add_constant(X_train.copy()) # 複製一份X_train的資料並新增截距項
models = sm.OLS(Y_train, X_train2) # 配適模型
result = models.fit() # 取得模型的預測結果
print(result.summary()) # 印出模型預測結果的報表

                            OLS Regression Results                            
Dep. Variable:                   實銷貨量   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     856.4
Date:                Tue, 27 Aug 2019   Prob (F-statistic):               0.00
Time:                        15:32:37   Log-Likelihood:                 64690.
No. Observations:              312051   AIC:                        -1.291e+05
Df Residuals:                  311901   BIC:                        -1.275e+05
Df Model:                         149                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    2

### 展示模型中各個解釋變數的迴歸係數(包含截距項)

In [15]:
displayModelCoef('線性迴歸', list(X_train), linreg.intercept_, linreg.coef_)

,變數名稱,線性迴歸_迴歸係數
0,截距項,168117866619.4556
1,實銷貨量_延遲2期,0.2695
2,實銷貨量_延遲3期,0.2567
3,實銷貨量_延遲4期,0.2474
4,含稅總成本_延遲2期,0.0492
5,含稅總成本_延遲3期,0.0429
6,含稅總成本_延遲4期,0.0190
7,成本合計_延遲2期,-0.1174
8,成本合計_延遲3期,-0.0919
9,成本合計_延遲4期,-0.0737


### 展示模型預測測試集成效的各項指標和模型預測測試集的MAPE分布

In [ ]:
# 現在的預測方法是什麼？
modelName = 'Linear Regression'

## 取得Y的實際值和預測值 ##
Y_real = Y_test.values
# 由於有對Y_train進行縮放，需要將預測值還原
Y_pred = inverseMinMaxByProduct(linreg.predict(X_test), Y_train_max_by_product_list, Y_train_min_by_product_list)
#Y_pred = linreg.predict(X_test)

displayForecastAccuracy(modelName, Y_real, Y_pred, X_test) # 展示模型預測測試集成效的各項指標
draw_forecast_MAPE_distribution(modelName, Y_real, Y_pred, 25000, 5000) # 畫出模型預測測試集的MAPE分布

 ## 模型2 - Lasso Regression (L1 Regularization) (誤差加上alpha乘以權重絕對值的總和)
 ### 原本是採用sklearn.linear_model的Lasso建模，現在改用sklearn.linear_model的LassoCV，這樣就可以透過機器自動做交叉驗證挑選最好的alpha值，目前cv=5，即5-fold cross-validation

### 配適模型

In [ ]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv = 5) # 設定regressor
lasso.fit(X_train, Y_train) # 配適模型

### 展示模型中各個解釋變數的迴歸係數(包含截距項)

In [ ]:
displayModelCoef('Lasso迴歸', list(X_train), lasso.intercept_, lasso.coef_) # 列出該模型各個變數的迴歸係數

### 展示模型預測測試集成效的各項指標和模型預測測試集的MAPE分布

In [ ]:
modelName = 'Lasso Regression' # 模型名稱

## 取得Y的實際值和預測值 ##
Y_real = Y_test.values
# 由於有對Y_train進行縮放，需要將預測值還原
Y_pred = inverseMinMaxByProduct(lasso.predict(X_test), Y_train_max_by_product_list, Y_train_min_by_product_list)
#Y_pred = lasso.predict(X_test)

displayForecastAccuracy(modelName, Y_real, Y_pred, X_test) # 展示模型預測測試集成效的各項指標
draw_forecast_MAPE_distribution(modelName, Y_real, Y_pred, 25000, 5000) # 畫出模型預測測試集的MAPE分布

## 模型3 - Ridge Regression (L2 Regularization) (誤差加上alpha乘以權重平方的總和)

### 原本是採用sklearn.linear_model的Ridge建模，現在改用sklearn.linear_model的RidgeCV，這樣就可以透過機器自動做交叉驗證挑選最好的alpha值，目前cv=5，即5-fold cross-validation

### 配適模型

In [ ]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(cv = 5) # 設定regressor
ridge.fit(X_train, Y_train) # 配適模型

### 展示模型中各個解釋變數的迴歸係數(包含截距項)

In [ ]:
displayModelCoef('Lasso迴歸', list(X_train), ridge.intercept_, ridge.coef_)

### 展示模型預測測試集成效的各項指標和模型預測測試集的MAPE分布

In [ ]:
modelName = 'Ridge Regression' # 模型名稱

## 取得Y的實際值和預測值 ##
Y_real = Y_test.values
# 由於有對Y_train進行縮放，需要將預測值還原
Y_pred = inverseMinMaxByProduct(ridge.predict(X_test), Y_train_max_by_product_list, Y_train_min_by_product_list)
#Y_pred = ridge.predict(X_test)

displayForecastAccuracy(modelName, Y_real, Y_pred, X_test) # 展示模型預測測試集成效的各項指標
draw_forecast_MAPE_distribution(modelName, Y_real, Y_pred, 25000, 5000) # 畫出模型預測測試集的MAPE分布

# **附錄 (資料操作相關的補充 & 最後沒有要實作的程式碼)

## 附錄1 - 從資料集中篩選出「存活產品」的資料，以及計算其相關的統計資訊 (optional)

### 建立存活產品清單 (目前對存活產品的定義：2017年最後6個月份的實銷貨量都有超過100以上)

In [ ]:
data_secondHalfof2017 = data[data['年/月'].isin(['2017/07', '2017/08', '2017/09', '2017/10', '2017/11', 
                                               '2017/12'])].reset_index(drop = True) # 從原始資料集中取得2017下半年的資料

survivingProduct_list = [] # 建立存活產品清單

# 篩選符合條件的存活產品
for i in range(0,7257):
    counter = 0
    for j in range(0,6):
        if data_secondHalfof2017['實銷貨量'][i * 6 + j] > 100:
            counter += 1
        if j == 5:
            if counter == 6:
                survivingProduct_list.append(data_secondHalfof2017.loc[i*6+j]['產品名稱'])

#print('2017年最後6個月份的實銷貨量都有超過100以上的存活產品清單：\n' + str(survivingProduct_list))
print('2017年最後6個月份的實銷貨量都有超過100以上的存活產品數量為：' + str(len(survivingProduct_list)))

### 從資料集中取出產品名稱有列在存活產品清單的資料

In [ ]:
data_survivingProduct = data[data['產品名稱'].isin(survivingProduct_list)].reset_index(drop = True)

### 2014/1 ~ 2017/12 存活產品 VS 整體產品 總實銷貨量

In [ ]:
print('整體產品2014/1至2017/12間的總實銷貨量：', data['實銷貨量'].sum())
print('存活產品2014/1至2017/12間的總實銷貨量：', data_survivingProduct['實銷貨量'].sum())
print('存活產品2014/1至2017/12間的總實銷貨量佔整體產品的比例：', 
      (data_survivingProduct['實銷貨量'].sum() / data['實銷貨量'].sum()))

### 2017下半年 存活產品 VS 整體產品 總實銷貨量

In [ ]:
print('整體產品2017下半年的總實銷貨量：', data_secondHalfof2017['實銷貨量'].sum())
print('存活產品2017下半年的總實銷貨量：', 
      data_secondHalfof2017[data_secondHalfof2017['產品名稱'].isin(survivingProduct_list)]['實銷貨量'].sum())
print('存活產品2017下半年的總實銷貨量佔整體產品的比例：',
      (data_secondHalfof2017[data_secondHalfof2017['產品名稱'].isin(survivingProduct_list)]['實銷貨量'].sum() / 
       data_secondHalfof2017['實銷貨量'].sum()))

### 用只含有存活產品的資料集取代目前在用的資料

In [ ]:
data = data_survivingProduct
data

## 附錄2 - 畫出模型預測測試集各月份的MAPE (長條圖)
### 目前測試集只有一個月份因此這個部分用不到

In [ ]:
# 在stat_adj中加上月分的欄位
stat_adj['月'] = data[(data['年'] == 2017) & (data['月'] >= 7) & (data['月'] <= 12)]['月'].reset_index(drop = True)

ax = stat_adj.groupby(['月']).mean()['mape'].plot.bar(
    figsize = (15, 7),
    fontsize = 16
)
ax.set_title('「MA3」預測測試集各月份的MAPE', fontsize = 20)
ax.set_xlabel('月份', fontsize = 20)
ax.set_ylabel('MAPE', fontsize = 20)
plt.xticks(rotation = 0)
for i, v in enumerate(stat_adj.groupby(['月']).mean()['mape'].values):
    ax.text(i, v, int(v), horizontalalignment='center', fontsize = 20)

## 附錄3 - 另一種透過交叉驗證找正規化模型懲罰項的方法

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
lasso = Lasso()
parameters = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring = 'neg_mean_squared_error', cv = 5)
lasso_regressor.fit(X_train, Y_train)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

lasso = Lasso(alpha = lasso_regressor.best_params_)
lasso.fit(X_train, Y_train)